# LSTM

In [33]:
import numpy as np
import pandas as pd
import random
from sklearn.ensemble import RandomForestClassifier
from src.utils import preprocess
from tqdm import tqdm

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense
from keras.layers import Embedding
from keras.layers import LSTM
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import losses

import time
random.seed(7)

%matplotlib inline
import matplotlib.pyplot as plt

## Loading data

In [13]:
# Open files
Data_X_train = pd.read_csv('data/challenge_fichier_dentrees_dentrainement_challenge_nba/train.csv')
Data_Y_train = pd.read_csv('data/challenge_fichier_de_sortie_dentrainement_challenge_nba.csv', sep=';')
X_test = pd.read_csv('data/challenge_fichiers_dentrees_de_test_challenge_nba/test.csv')

**Data preprocessing : split train/validation and remove ID**

In [23]:
X_train, Y_train, X_val, Y_val = preprocess(Data_X_train, Data_Y_train, 0.7, True)

## Model evaluation

In [26]:
nb_games_train = len(X_train)
nb_games_val = len(X_val)
X_train = X_train.reshape((nb_games_train, 11, 1440, 1), order = 'F')
X_val = X_val.reshape((nb_games_val, 11, 1440, 1), order = 'F')
Y_train = np_utils.to_categorical(Y_train, 2)
Y_val = np_utils.to_categorical(Y_val, 2)

In [38]:
lstm_model = Sequential()
lstm_model.add(LSTM(50, input_shape = (1, 1440)))
lstm_model.add(Dense(units = 2, 
                activation='softmax'))

#Construct Loss
conv_model.compile(loss = losses.categorical_crossentropy,
                     optimizer = 'adam',
                     metrics = ['accuracy'])

#Train
start = time.time()
training = conv_model.fit(X_train, Y_train, epochs = 100, batch_size = 32, verbose = False)
end = time.time()

print('TRAINING')
print('Loss at last iteration: {}'.format(training.history['loss'][-1]))
print('Accuracy at last iteration: {}'.format(training.history['acc'][-1]))
print('Training time: {}'.format(end - start))

print('\nTEST')
evalu = conv_model.evaluate(X_val, Y_val)
print('Loss: {}'.format(evalu[0]))
print('Accuracy: {}'.format(evalu[1]))

TRAINING
Loss at last iteration: 0.49186616133901456
Accuracy at last iteration: 0.7604225832170401
Training time: 5921.002259731293

TEST
3773/3773 [==============================] - ETA: 1: - ETA: 43s - ETA: 33 - ETA: 29 - ETA: 26 - ETA: 24 - ETA: 22 - ETA: 21 - ETA: 20 - ETA: 19 - ETA: 19 - ETA: 18 - ETA: 18 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ET

In [39]:
conv_model.save('models/lstm1.h5')